<h2>Text Analysis with Scikit-learn</h2>

Thanks to everyone for coming today. Excited to be here and talk about a neat topic I really enjoy, which is text analysis with Python, specifically with scikit-learn. While I have been able to use this at work occassionally, I am definitely not an Natural Language Processing expert. However, one of the cool things about the python scientific computing ecosystem is that you can *not* be an expert and still do some neat stuff.

Title slide. Contact info. github info.

::get slides from work::

Text analysis is the extraction of information from unstructured (in most cases) text. This is difficult because computer only understand numbers. So in order to analyze text with computers, we have to do some kind of processing to it. There are two basic ways to approach text analysis: look for patterns in the text (these words often appear together/near each other (multi-word term), these words are often used in the same context but never appear together (synonyms), etc.); or you can convert the text to numbers and do math on them. 

The technique we're going to discuss today is called Term Frequency-Inverse Document Frequency (TF-IDF), and it is a fairly common method in the area of Information Retrieval (IR) for comparing and searching documents. What we're going to end up doing is taking our big set of documents, called the corpus, and these can be any type of document: news articles, emails, online comments (although the longer the documents the better the results), and we're going to turn them into matrices (number), and then we're going to do some pretty basic math on them. That math is going to help us define what's called a distance metric between any two documents, which we can then use for comparisions and searches. Ok, but first, let's think about this problem intuitively. 

Let's say we have three documents. Our documents are just going to be sentences. 
<li>The dog is jumping over the fence.</li>
<li>The dog is climbing up the fence.</li>
<li>The cat is sitting on the window.</li>
So how similar/disimilar are these sentences? Well, that's a little hard to answer right now because that is trying to quantify something about the text. What about a difference question: which two sentences are the most similar? The first two?  Why? They share a lot of the same words, so we can think of them as being similar in their contents based on the inclusion of identical words. 

::Add another sentence and point out which words end up being the important ones.::
<li>A dog was sitting in this window.</li>

So this is good. We could define a similarity metric as the number of words two sentences have in common. But is that good enough? What about these sentences?
<li>A dog was jumping over this fence and then ran over to the next yard so very quickly.</li>
<li>A cat was leaning over this edge and then jumped over to the next table so very quickly.</li>
They have many more words in common than our first two sentences, so are they more similar? Probably not. But this is function of the length of the sentences, so let's try to normalize by dividing by the number of words in the sentence. This will give us a percent of similarity. Well it turns out these longer sentences still get a higher similarity score. So what's the problem? Intiutively we know the first two sentences are similar, but it seems that using numbers suggests the latest two sentences are more similar. So what's the problem? The problem is that we're considering all the words equal, when, for purposes of distiguishing sentences from one another, they are not equal. Why not? 
<li>A dog was jumping over this fence and then ran over to the next yard so very quickly.</li>
<li>A cat was leaning over this fence and then jumped over to the next yard so very quickly.</li>
<li>A squirrel was leaping over this branch and then scampered over to the next tree so very quickly.</li>
<li>A bird was peering over this branch and then flew over to the next tree.</li>
If we were going to distiguish between these sentences, any word that appears in all the sentences is worthless. It doesn't help us to determine if any pair of sentences is similar or disimilar because it's common to all of them. It's almost a characteristic of a sentence in general, as opposed to a charateristics of a particular sentence. So we don't want to use words that appear in all sentences. We either throw them out or weight them very low (more on that later). What about words that appear in a lot but not all sentences? "so very quickly" appears in all but one sentence....
Somewhat useful, but not super useful.
Tree, branch, fence, yard: appear in some, but not all or most, so they are usefula nd will get weighted the highest.
This is the Inverse Document Frequency part of the TF-IDF. The more documents a word appears in the, the less useful it is in distinguishing between the documents, so it get weighted less. The Inverse essentially means we divide by the number of documents the word (or term) appears in. 
For the first part, the Term Frequency, we're simply going to calculate the frequency of each term in each document by counting the number of times it appears in the document and dividing by the total number of words in the documnets. The assumption here is that the more often a word appears in a document, the more closely it represents the topic of the document.


Ok, so let's actually calculate some TF-IDFs by hand, and then we'll do it in Python.
::example with a few sentences::
::create list of all the words. Count number of docs each word appear in. create vector for each doc. Calculate Euclidean distance::
Show distances between pairs of sentences. Show the dog and cat are similar and the squirrel and bird are similar.




<div style="margin-top:200px"/>

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd

###### Example 1: Short Sentences

In [2]:
short_sentences = ['The dog is jumping over the fence.', 
                   'The dog is climbing up the fence.',
                   'The cat is sitting on the window.']

In [3]:
# Create an instance of the CountVectorizer.
cv = CountVectorizer()

# Create a count matrix from the list of documents.
count_matrix = cv.fit_transform(short_sentences)

In [28]:
# Create a dataframe out of the count matrix with labels and columns names for easier reading.
short_sent_df = pd.DataFrame(count_matrix.todense(), index=short_sentences, columns=sorted(cv.vocabulary_))
short_sent_df

,cat,climbing,dog,fence,is,jumping,on,over,sitting,the,up,window
The dog is jumping over the fence.,0,0,1,1,1,1,0,1,0,2,0,0
The dog is climbing up the fence.,0,1,1,1,1,0,0,0,0,2,1,0
The cat is sitting on the window.,1,0,0,0,1,0,1,0,1,2,0,1


<div style="margin-top:200px"/>

###### Example 2: Long sentences

In [150]:
long_sentences = ['A dog was jumping over this fence and then ran over to the next yard so very quickly.',
                  'A cat was leaning over this fence and then jumped over to the next yard so very quickly.',
                  'A squirrel was leaping over this branch and then scampered over to the next tree so very quickly.',
                  'A bird was peering over this branch and then flew over to the next tree.']

In [140]:
long_sentences = ['dog dog dog dog dog dog dog dog something else.',
                  'A cat was leaning over this fence and then jumped over to the next yard so very quickly.',
                  'A squirrel was leaping over this branch and then scampered over to the next tree so very quickly.',
                  'A bird was peering over this branch and then flew over to the next tree.']

In [151]:
# Create an instance of the CountVectorizer.
cv = CountVectorizer()

# Create a count matrix from the list of documents.
long_count_matrix = cv.fit_transform(long_sentences)

In [152]:
# Create a dataframe out of the count matrix with labels and columns names for easier reading.
long_sent_df = pd.DataFrame(long_count_matrix.todense(), index=long_sentences, columns=sorted(cv.vocabulary_))
long_sent_df

,and,bird,branch,cat,dog,fence,flew,jumped,jumping,leaning,...,so,squirrel,the,then,this,to,tree,very,was,yard
A dog was jumping over this fence and then ran over to the next yard so very quickly.,1,0,0,0,1,1,0,0,1,0,...,1,0,1,1,1,1,0,1,1,1
A cat was leaning over this fence and then jumped over to the next yard so very quickly.,1,0,0,1,0,1,0,1,0,1,...,1,0,1,1,1,1,0,1,1,1
A squirrel was leaping over this branch and then scampered over to the next tree so very quickly.,1,0,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
A bird was peering over this branch and then flew over to the next tree.,1,1,1,0,0,0,1,0,0,0,...,0,0,1,1,1,1,1,0,1,0


In [153]:
# Create an instance of a TfidfTransformer.
tfidf = TfidfTransformer(norm='l1', use_idf=False, smooth_idf=False)

# Created a weighted matrix.
weighted_matrix = tfidf.fit_transform(long_count_matrix)

In [154]:
weighted_matrix

<4x27 sparse matrix of type '<type 'numpy.float64'>'
	with 61 stored elements in Compressed Sparse Row format>

In [155]:
# Create a dataframe out of the count matrix with labels and columns names for easier reading.
long_sent_weighted_df = pd.DataFrame(weighted_matrix.todense(), index=[s.split()[1] for s in long_sentences], columns=sorted(cv.vocabulary_))
long_sent_weighted_df

,and,bird,branch,cat,dog,fence,flew,jumped,jumping,leaning,...,so,squirrel,the,then,this,to,tree,very,was,yard
dog,0.058824,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.058824,0.000000,...,0.058824,0.000000,0.058824,0.058824,0.058824,0.058824,0.000000,0.058824,0.058824,0.058824
cat,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,...,0.058824,0.000000,0.058824,0.058824,0.058824,0.058824,0.000000,0.058824,0.058824,0.058824
squirrel,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.000000
bird,0.071429,0.071429,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.071429,0.071429,0.071429,0.071429,0.071429,0.000000,0.071429,0.000000


In [156]:
long_sent_weighted_df['quickly']

dog         0.058824
cat         0.058824
squirrel    0.058824
bird        0.000000
Name: quickly, dtype: float64

In [165]:
1/14.0

0.07142857142857142

In [164]:
pd.np.sum(long_count_matrix.todense(), axis=1)

matrix([[17],
        [17],
        [17],
        [14]], dtype=int64)

In [158]:
# Print the number of words in each sentence.
for s in long_sentences:
    print(s.split()[1] + ' ' + str(len(s.split())))

dog 18
cat 18
squirrel 18
bird 15


In [159]:
zip(sorted(cv.vocabulary_), tfidf.idf_)

TypeError: zip argument #2 must support iteration

<div style="margin-top:200px"/>

###### Example 3: An interesting one

In [151]:
# Create an instance of the CountVectorizer.
cv = CountVectorizer()

# Create a count matrix from the list of documents.
long_count_matrix = cv.fit_transform(long_sentences)

In [152]:
# Create a dataframe out of the count matrix with labels and columns names for easier reading.
long_sent_df = pd.DataFrame(long_count_matrix.todense(), index=long_sentences, columns=sorted(cv.vocabulary_))
long_sent_df

,and,bird,branch,cat,dog,fence,flew,jumped,jumping,leaning,...,so,squirrel,the,then,this,to,tree,very,was,yard
A dog was jumping over this fence and then ran over to the next yard so very quickly.,1,0,0,0,1,1,0,0,1,0,...,1,0,1,1,1,1,0,1,1,1
A cat was leaning over this fence and then jumped over to the next yard so very quickly.,1,0,0,1,0,1,0,1,0,1,...,1,0,1,1,1,1,0,1,1,1
A squirrel was leaping over this branch and then scampered over to the next tree so very quickly.,1,0,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
A bird was peering over this branch and then flew over to the next tree.,1,1,1,0,0,0,1,0,0,0,...,0,0,1,1,1,1,1,0,1,0


In [153]:
# Create an instance of a TfidfTransformer.
tfidf = TfidfTransformer(norm='l1', use_idf=False, smooth_idf=False)

# Created a weighted matrix.
weighted_matrix = tfidf.fit_transform(long_count_matrix)

In [154]:
weighted_matrix

<4x27 sparse matrix of type '<type 'numpy.float64'>'
	with 61 stored elements in Compressed Sparse Row format>

In [155]:
# Create a dataframe out of the count matrix with labels and columns names for easier reading.
long_sent_weighted_df = pd.DataFrame(weighted_matrix.todense(), index=[s.split()[1] for s in long_sentences], columns=sorted(cv.vocabulary_))
long_sent_weighted_df

,and,bird,branch,cat,dog,fence,flew,jumped,jumping,leaning,...,so,squirrel,the,then,this,to,tree,very,was,yard
dog,0.058824,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.058824,0.000000,...,0.058824,0.000000,0.058824,0.058824,0.058824,0.058824,0.000000,0.058824,0.058824,0.058824
cat,0.058824,0.000000,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,...,0.058824,0.000000,0.058824,0.058824,0.058824,0.058824,0.000000,0.058824,0.058824,0.058824
squirrel,0.058824,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.058824,0.000000
bird,0.071429,0.071429,0.071429,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.071429,0.071429,0.071429,0.071429,0.071429,0.000000,0.071429,0.000000


In [2]:
import requests
from bs4 import BeautifulSoup

In [41]:
def get_song_links(url):
    """Gets song titles and URLs for the lyrics page for a www.songlyrics.com top-songs page."""
    soup = BeautifulSoup(requests.get(url).content)
    trs = soup.find_all('table', class_='tracklist')[0].find_all('tr')
    song_links = {}
    for row in trs[1:]:
        tag = row.find_all('td')[-1]
        a_tag = tag.a
        song_links[a_tag['title']] = a_tag['href']
    return song_links

In [42]:
def get_lyrics(url):
    """Scrapes the lyrics from a www.songlyrics.com song page."""
    soup = BeautifulSoup(requests.get(url).content)
    return soup.find_all('p', attrs={'id':'songLyricsDiv'})[0].get_text('\n', strip=True)

In [45]:
# Define variables
base_url = 'http://www.songlyrics.com/news/top-songs/'
start_year = 2011
end_year = 1990

In [46]:
# Dict to hold song titles -> link to lyrics page.
song_links = {}
cur_year = start_year

# Loop through years and scrape links to lyric pages for each year's top 100 songs.
while cur_year >= end_year:
    url = base_url + str(cur_year)
    song_links.update(get_song_links(url))
    cur_year -= 1

In [51]:
# Dict to hold song titles -> lyrics
song_lyrics = {}
errors = {}
# Visit the lyric pages for each song in song_links and scrape the lyrics.
for title, url in song_links.iteritems():
    try:
        song_lyrics[title] = get_lyrics(url)
    except Exception as e:
        errors[title] = e

In [52]:
tup = song_lyrics.items()[2]
print(tup[0] +'\n'+ tup[1])

Who Says you can't Go Home Lyrics  Bon Jovi
I spent twenty years tryin' to get out of this place
I was lookin' for somethin' I couldn't replace
I was runnin' away from the only thing I've ever known
And like a blind dog without a bone
I was a gypsy lost in the twilight zone
I hijacked a rainbow and crashed into a pot of gold
I've been there, done that, now I ain't lookin' back
And the seeds I've sown, savin' dimes
Spendin' too much time on the telephone
Who says you can't go home?
Who says you can't go home?
There's only one place that call me one of their own
Just a hometown boy, born a rollin' stone
Who says you can't go home?
Who says you can't go back?
Been all around the world and as a matter of fact
There's only one place left, I want to go
Who says you can't go home? It's alright
It's alright, it's alright, it's alright, it's alright
I went as far as I could, I tried to find a new face
There isn't one of these lines that I would erase
I left a million mile of memories on that ro

In [53]:
len(song_lyrics)

2046

In [56]:
[k for k in song_lyrics.keys() if 'matthews' in k.lower()]

['The Space Between Lyrics  Dave Matthews Band']

In [57]:
errors

{'1 Thing Lyrics  Amerie': IndexError('list index out of range'),
 'Any Time, Any Place / And On And On Lyrics  Janet Jackson': IndexError('list index out of range'),
 'Before You Walk Out Of My Life / Like This And Like That Lyrics  Monica': IndexError('list index out of range'),
 'Big Poppa / Warning Lyrics  Notorious B.I.G.': IndexError('list index out of range'),
 'Boombastic / In The Summertime Lyrics  Shaggy': IndexError('list index out of range'),
 "C'mon And Get My Love Lyrics  D-Mob With Cathy Dennis": requests.exceptions.InvalidSchema(u"No connection adapters were found for ' http://www.songlyrics.com/d-mob-introducing-cathy-dennis/c-mon-and-get-my-love-lyrics/'"),
 'Dear Mama / Old School Lyrics Tupac': IndexError('list index out of range'),
 'Do For Love Lyrics Tupac': requests.exceptions.InvalidSchema(u"No connection adapters were found for ' http://www.songlyrics.com/tupac/do-for-love-lyrics/'"),
 "Don't Know Much Lyrics  Linda Ronstadt and Aaron Neville": IndexError('lis

In [58]:
len(errors)

35

In [59]:
import json

In [ ]:
# Save song lyrics.
json.dump(song_lyrics, open('song_lyrics.json', 'wt'), encoding='utf8')

http://nlp.stanford.edu/IR-book/html/htmledition/document-and-query-weighting-schemes-1.html
TextBlob